# Post-Processing:

This file was created to run a post-process before the geolocation took place.
The script takes the csv files created in the "01_Pre_process" script and shapes the addresses with the desired form, it corrects recurrent misspellings and mistakes from the tesseract model. To obtain relatively correct outcomes, this script has required considerable time to adjust the shape and correct for misspellings and other mistakes from the OCR.

This script has been deprecated after changing the approach to the one used in the script "02_AI_OCR". The use of the AI model to read and shape the pdf images follows the structure of  the paper "MULTIMODAL LLMS FOR OCR, OCR POST-CORRECTION, AND NAMED ENTITY RECOGNITION IN HISTORICAL DOCUMENTS" from Gavin Greif, Niclas Griesshaber and Robin Greif and has drastically reduced time in human correction.


In [ ]:
# Libraries

import pandas as pd
import numpy as np
import re
import os
from google import genai
import csv
import io
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [ ]:
# mount google colab
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# Include here your working directory
%cd /content/drive/MyDrive/2_FEINA/10_HU/01_TASKS/01_DIRECTORIES


/content/drive/MyDrive/2_FEINA/10_HU/01_TASKS/01_DIRECTORIES


In [ ]:
# Include here the city and state
city = "Kansas City"
state = "Missouri"

city_df = pd.read_csv(f"files/{city}/{city}.csv")

In [ ]:
city_df = pd.DataFrame(city_df)
city_df

,Page Number,Text,Company_Type,City,County,Company_Name,Address,Building,Full_Address
0,1,"KANSAS CITY , 248,381 Attorney",NaN,Kansas City,Missouri,"KANSAS CITY ,","248,381 Attorney",NaN,"248,381 Attorney, Kansas City, Missouri"
1,1,"JNO. A, EATON, Attorney, 800 U. 5.",Attorney,Kansas City,Missouri,"JNO. A, EATON, Attorney,",800 U. 5.,NaN,"800 U. 5., Kansas City, Missouri"
2,1,Mex. Trust building. Special attention,Attorney,Kansas City,Missouri,Mex. Trust building. Special attention,NaN,Trust building,"Trust building, Kansas City, Missouri"
3,1,given to Real Estate Ww,Attorney,Kansas City,Missouri,given to Real Estate Ww,NaN,NaN,NaN
4,1,Real Estate,Real Estate,Kansas City,Missouri,Real Estate,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
848,7,"L. R, Wright, 206 R. A. Long building.",Real Estate,Kansas City,Missouri,"L. R, Wright, 206 R. A. Long building.",NaN,206 R. A. Long building,"206 R. A. Long building, Kansas City, Missouri"
849,7,"M. B Wright, 919 Baltimore ave.",Real Estate,Kansas City,Missouri,"M. B Wright,",919 Baltimore ave.,NaN,"919 Baltimore ave., Kansas City, Missouri"
850,7,"Jamea R. Young, 335 Ridge pide.",Real Estate,Kansas City,Missouri,"Jamea R. Young,",335 Ridge pide.,NaN,"335 Ridge pide., Kansas City, Missouri"
851,7,"Zavala Land Inv. Co., 602 Grand Ave.",Real Estate,Kansas City,Missouri,"Zavala Land Inv. Co.,",602 Grand Ave.,NaN,"602 Grand Ave., Kansas City, Missouri"


In [ ]:
# REducing df to only relevant rows:
city_df = city_df.loc[128:1041].reset_index(drop=True)
# city_df = city_df.drop(city_df.index[137:213]).reset_index(drop=True)
city_df

,Page Number,Text
0,1,"KANSAS CITY (Jackson), 248,381"
1,1,dttorney;
2,1,"JNO. A, EATON, Attorney, 800 U. 5. &"
3,1,Mex. Trust bidg. Special attention
4,1,given to Real Estate Ww
...,...,...
909,7,"L. R, Wright, 206 R. A. Long bidg."
910,7,"M. B Wright, 919 Baltimore ave. |"
911,7,"Jamea R. Young, 335 Ridge pide."
912,7,"Zavala Land Inv. Co., 602 Grand Ave. |"


In [ ]:
# Making sure Attorney, Real state and other company types are written correctly:
def cleaning_company_types(df):
  #Will include different transformations of the words
  df['Text'] = df['Text'].str.replace(r'dttorney;', 'Attorney:', regex=True)
  df['Text'] = df['Text'].str.replace(r'atone', 'Attorney:', regex=True)
  df['Text'] = df['Text'].str.replace(r'veal Estate:', 'Real Estate::', regex=True)
  df['Text'] = df['Text'].str.replace(r'Real Batate:', 'Real Estate:', regex=True)
  df['Text'] = df['Text'].str.replace(r'Real Hetate:', 'Real Estate:', regex=True)
  df['Text'] = df['Text'].str.replace(r'Reat Batate:', 'Real Estate:', regex=True)
  df['Text'] = df['Text'].str.replace(r'Reali Estate:', 'Real Estate:', regex=True)
  df['Text'] = df['Text'].str.replace(r'Real Batate:', 'Real Estate:', regex=True)
  df['Text'] = df['Text'].str.replace(r'Land Companies:', 'Land Companies:', regex=True)
  df['Text'] = df['Text'].str.replace(r'Lond Compantes:', 'Land Companies:', regex=True)
  df['Text'] = df['Text'].str.replace(r'Land port: :', 'Land Script:', regex=True)
  df['Text'] = df['Text'].str.replace(r'4bstract Compunics:', 'Abstract Companies:', regex=True)
  df['Text'] = df['Text'].str.replace(r'Abstracts :', 'Abstracts:', regex=True)

#
  return df

In [ ]:
city_comp = cleaning_company_types(city_df)
city_comp

,Page Number,Text
0,1,"KANSAS CITY (Jackson), 248,381"
1,1,Attorney:
2,1,"JNO. A, EATON, Attorney, 800 U. 5. &"
3,1,Mex. Trust bidg. Special attention
4,1,given to Real Estate Ww
...,...,...
909,7,"L. R, Wright, 206 R. A. Long bidg."
910,7,"M. B Wright, 919 Baltimore ave. |"
911,7,"Jamea R. Young, 335 Ridge pide."
912,7,"Zavala Land Inv. Co., 602 Grand Ave. |"


In [ ]:
def company_types(df):
  df['Company_Type'] = ''

  # Types of companies
  lst_types = ['Attorney:', 'Real Estate:', 'Land Companies:', "Abstract Companies:","Abstractors:","Abstracts", "Land Script:"]

  # Iterate through each row in the DataFrame
  new_type = None  # To keep track of the last detected company type

  for index, value in df['Text'].items():
      # Check if any keyword in lst_types is present in the text
      match = next((word for word in lst_types if word in value), None)

      if match:
          new_type = match  # Store the detected company type
          df.loc[index, 'Company_Type'] = new_type
      else:
          df.loc[index, 'Company_Type'] = new_type  # Carry forward the last detected type

  # Remove the ":" from the company type
  df['Company_Type'] = df['Company_Type'].str.replace(r':', '', regex=True)

  return df

city_comp = company_types(city_comp)
city_comp

,Page Number,Text,Company_Type
0,1,"KANSAS CITY (Jackson), 248,381",None
1,1,Attorney:,Attorney
2,1,"JNO. A, EATON, Attorney, 800 U. 5. &",Attorney
3,1,Mex. Trust bidg. Special attention,Attorney
4,1,given to Real Estate Ww,Attorney
...,...,...,...
909,7,"L. R, Wright, 206 R. A. Long bidg.",Real Estate
910,7,"M. B Wright, 919 Baltimore ave. |",Real Estate
911,7,"Jamea R. Young, 335 Ridge pide.",Real Estate
912,7,"Zavala Land Inv. Co., 602 Grand Ave. |",Real Estate


In [ ]:
# some rows contain small particles, like bway, or bldg which should not be a row in itself,
# bu t part of the previous row

def short_rows(df):
  short_rows = df[df['Text'].str.len() < 10]
  for i in short_rows.index:
    if i > 0:
      # it appends the text of the row i to the previous row i-1
      df.loc[i-1, 'Text'] = df.loc[i-1, 'Text'] + ' ' + df.loc[i, 'Text']

  # deletes short rows from the df:
  df.drop(short_rows.index, inplace=True)
  df.reset_index(drop=True, inplace=True)

  return df
# append short rows to precious rows
city_df_nosr = short_rows(city_comp)
city_df_nosr


,Page Number,Text,Company_Type
0,1,"KANSAS CITY (Jackson), 248,381 Attorney:",None
1,1,"JNO. A, EATON, Attorney, 800 U. 5. &",Attorney
2,1,Mex. Trust bidg. Special attention,Attorney
3,1,given to Real Estate Ww,Attorney
4,1,Real Estate:,Real Estate
...,...,...,...
848,7,"L. R, Wright, 206 R. A. Long bidg.",Real Estate
849,7,"M. B Wright, 919 Baltimore ave. |",Real Estate
850,7,"Jamea R. Young, 335 Ridge pide.",Real Estate
851,7,"Zavala Land Inv. Co., 602 Grand Ave. |",Real Estate


In [ ]:
def post_process(df, cityname):
  df['City'] = cityname
  df['State'] = state
  df['Company_Name'] = ''
  df['Address'] = ''
  df['Building'] = ''

  #
## COMMON MISTAKES:

# Remove all text inside parenthesis:
  df['Text'] = df['Text'].str.strip().str.replace(r'\(.*?\)', '', regex=True)

# Delete all special characters I find
  df['Text'] = df['Text'].str.replace(r'\(See adv.\)|\(Bee, adv.\)|\(See\)', ' ', regex=True)
  df['Text'] = df['Text'].str.strip().str.replace(r'[:;!?"¨{}«°()_\-\|º=¢<>&\/*€%$©@\x0c¥§]+', '', regex=True)

  # Recurrent errors
  df['Text'] = df['Text'].str.strip().str.replace(r'\b(Bmpirs|Einptre|Emptre)\b', 'Empire', regex=True)
  df['Text'] = df['Text'].str.strip().str.replace(r'\b(Nassaa|Naasau)(\,?)\b', 'Nassau', regex=True)
  df['Text'] = df['Text'].str.strip().str.replace(r'\b(Amstrdm)(\.?)\b', 'Amsterdam', regex=True)

  df['Text'] = df['Text'].str.strip().str.replace(r'\b(Maiestic|Mujestic|Maseatic|Mafestic)(\,?)', 'Majestic', regex=True)
  df['Text'] = df['Text'].str.strip().str.replace(r'\b(Bk|Bank\.)\b', 'Bank', regex=True)
  df['Text'] = df['Text'].str.strip().str.replace(r'\b(Nat\'l|Nat\.)\b', 'National', regex=True)
  df['Text'] = df['Text'].str.strip().str.replace(r'\b(Bhort)(\,?)\b', 'National', regex=True)
  df['Text'] = df['Text'].str.strip().str.replace(r'8t\.?', 'st', regex=True)


  df['Text'] = df['Text'].str.strip().str.replace(r'\b(Blectric)\b', 'Electric', regex=True)
  df['Text'] = df['Text'].str.strip().str.replace(r'\b(Com- mercer|Con merce|Coommerce)\b', 'Commerce', regex=True)

  df['Text'] = df['Text'].str.strip().str.replace(r'\b(collces)\b', 'College', regex=True)
  df['Text'] = df['Text'].str.strip().str.replace(r'\b(Sco)\b', 'Scollard', regex=True)
  df['Text'] = df['Text'].str.strip().str.replace(r'\b(Fleld)\b', 'Field', regex=True)

  df['Text'] = df['Text'].str.strip().str.replace(r'\b(Falace|Palnce)\b', 'Palace', regex=True)
  df['Text'] = df['Text'].str.strip().str.replace(r'\b(Rockefellor)\b', 'Rockefeller', regex=True)
  df['Text'] = df['Text'].str.strip().str.replace(r'\b(Tr\.|Tr)\b', 'Trade', regex=True)
  df['Text'] = df['Text'].str.strip().str.replace(r'\b(8rd|Srd)\b', '3rd', regex=True)
  df['Text'] = df['Text'].str.strip().str.replace(r'\b(mkt|Mkt|mkt\.|Mkt\.)\b', 'Market', regex=True)
  df['Text'] = df['Text'].str.strip().str.replace(r'\b(Willlamson|mson|Wmason|Wililamson|Willlamsor|Wmeon|Wlllam- ena|Wmso|Wmson|Wmaon)(\.?)\b', 'Williamson', regex=True)
  df['Text'] = df['Text'].str.strip().str.replace(r'\b(Lomcke|Lemcko)\b', 'Lemcke', regex=True)
  df['Text'] = df['Text'].str.strip().str.replace(r'\b(Lite)\b', 'Life', regex=True)
  df['Text'] = df['Text'].str.strip().str.replace(r'\b(Tryo|Try)\b', 'Life', regex=True)


  df['Text'] = df['Text'].str.strip().str.replace(r'\b(Cawker)\b', 'Caswell', regex=True)
  df['Text'] = df['Text'].str.strip().str.replace(r'\b(Dougias)\b', 'Douglas', regex=True)

  df['Text'] = df['Text'].str.strip().str.replace(r'\b(Rtore)\b', 'Store', regex=True)

  df['Text'] = df['Text'].str.strip().str.replace(r'\b(Garfleld)\b', 'Garfield', regex=True)
  df['Text'] = df['Text'].str.strip().str.replace(r'\b(Broad|Brosa|Bdwy|Biwy|Bway)(\,?)\b', 'Broadway', regex=True)
  df['Text'] = df['Text'].str.strip().str.replace(r'\b(Bldg|Bldg\.|bldg|bldg\.)(\.?)\b', 'building', regex=True)
  df['Text'] = df['Text'].str.strip().str.replace(r'\b(Phoentx|Phoenfx)\b', 'Phoenix', regex=True)
  df['Text'] = df['Text'].str.strip().str.replace(r'\b(at\.)\b', 'st.', regex=True)
  df['Text'] = df['Text'].str.strip().str.replace(r'\b(aye|avé|av)(\.?)\b', 'ave', regex=True)

  df['Text'] = df['Text'].str.replace(r'', '', regex=True)
  df['Text'] = df['Text'].str.strip().str.replace(r'\b(Congrea|Congreas|Congresa|conaress)\b', 'Congress', regex = True)
  df['Text'] = df['Text'].str.strip().str.replace(r'\b(Columbla)\b', 'Columbia', regex = True)
  df['Text'] = df['Text'].str.strip().str.replace(r'\b(lat\,|iat\,|lst|lst\,|Iat|Iat\,|Ist)(\,?)\b', '1st', regex = True)
  df['Text'] = df['Text'].str.strip().str.replace(r'\b(Unlon|Unton|Unfon)\b', 'Union', regex = True)
  df['Text'] = df['Text'].str.strip().str.replace(r'\b(8imon)\b', 'Simon', regex = True)
  df['Text'] = df['Text'].str.strip().str.replace(r'\b(8tephenson)\b', 'Peter', regex = True)
  df['Text'] = df['Text'].str.strip().str.replace(r'\b(Petera)\b', '8th', regex = True)
  df["Text"] = df["Text"].str.strip().str.replace(r'\b(bid|bids|bide|bl|bidg|blilg|didg|blag|bdidg)(\.?)\b', 'building', regex=True)
  df["Text"] = df["Text"].str.strip().str.replace(r'\b(bik|bik\.)(\,?)', 'block', regex=True)
  df["Text"] = df["Text"].str.strip().str.replace(r'\b(Hppodrome)', 'Hippodrome', regex=True)
  df["Text"] = df["Text"].str.strip().str.replace(r'\b(Truat|Truant)(\,?)\b', 'Trust', regex=True)
  df["Text"] = df["Text"].str.strip().str.replace(r'\b(Chestaut)\b', 'Chestnut', regex=True)
  df["Text"] = df["Text"].str.strip().str.replace(r'\b(Weybossrt|Weyboaszet|Weybosaet|wey bosset|Weybossct|Weybdosset|Weybosart|Wevboaset)(\,?)\b', 'Weybosset', regex=True)
  df["Text"] = df["Text"].str.strip().str.replace(r'\b(Ranigan| Ranigean|Banizan)\b', 'Banigan', regex=True)
  df["Text"] = df["Text"].str.strip().str.replace(r'\b(Westminater|Weatminster|Westminater|Westmin ster|Weatminater)(\,?)\b', 'Westminster', regex=True)



  # df['Address'] = df['Address'].str.replace(r'\b(Bmpirs|Einptre|Emptre|Emp\.|Emp)\b', 'Empire', regex=True)
  df['Text'] = df['Text'].str.replace(r'(?i)\b(Bmpirs|Einptre|Emptre)\b', 'Empire', regex=True)
  df['Text'] = df['Text'].str.replace(r'Emp\.', 'Empire', regex=True)
  df['Text'] = df['Text'].str.replace(r'Empive Life', 'Empire Life', regex=True)


  df['Text'] = df['Text'].str.replace(r'Sth', '5th', regex=True)
  df['Text'] = df['Text'].str.replace(r'AY|AV', 'Av', regex=True)
  df['Text'] = df['Text'].str.replace(r'S4', '54', regex=True)


  # ADDRESS
  # regex to detect street pattern, after punctuation, and starts with number
  # can not contain the word "building"

  address_pattern = re.compile(r'\s*(\d+(?!.*building).*)')

  # Extract the pattern and store it in a new column called 'Address'
  df['Address'] = df['Text'].str.extract(address_pattern, expand=False)
  #

  # BUILDINGS:
  #  The pattern captures every charecter after the last  or , and the particle "building"
  building_pattern = re.compile(r'[,.]([^,]*\sbuilding|blk|building\.|block)', re.IGNORECASE)

  df['Building'] = df['Text'].str.extract(building_pattern, expand=False)

  # COMPANY NAME

  # Replace the extracted address with an empty string to isolate the Company name
  df['Company_Name'] = df['Text'].str.replace(address_pattern, '', regex=True)

  # Remove leading/trailing spaces and potential punctuation from the name
  df['Company_Name'] = df['Company_Name'].str.strip().str.replace(r'^[.,:;!?-]+$', '', regex=True)

  return df
city_post = post_process(city_df_nosr, city)
# city_post = post_process(city_df, city)
city_post

,Page Number,Text,Company_Type,City,County,Company_Name,Address,Building
0,1,"KANSAS CITY , 248,381 Attorney",None,Kansas City,Missouri,"KANSAS CITY ,","248,381 Attorney",NaN
1,1,"JNO. A, EATON, Attorney, 800 U. 5.",Attorney,Kansas City,Missouri,"JNO. A, EATON, Attorney,",800 U. 5.,NaN
2,1,Mex. Trust building. Special attention,Attorney,Kansas City,Missouri,Mex. Trust building. Special attention,NaN,Trust building
3,1,given to Real Estate Ww,Attorney,Kansas City,Missouri,given to Real Estate Ww,NaN,NaN
4,1,Real Estate,Real Estate,Kansas City,Missouri,Real Estate,NaN,NaN
...,...,...,...,...,...,...,...,...
848,7,"L. R, Wright, 206 R. A. Long building.",Real Estate,Kansas City,Missouri,"L. R, Wright, 206 R. A. Long building.",NaN,206 R. A. Long building
849,7,"M. B Wright, 919 Baltimore ave.",Real Estate,Kansas City,Missouri,"M. B Wright,",919 Baltimore ave.,NaN
850,7,"Jamea R. Young, 335 Ridge pide.",Real Estate,Kansas City,Missouri,"Jamea R. Young,",335 Ridge pide.,NaN
851,7,"Zavala Land Inv. Co., 602 Grand Ave.",Real Estate,Kansas City,Missouri,"Zavala Land Inv. Co.,",602 Grand Ave.,NaN


In [ ]:
def full_address(df):
  df['Full_Address'] = ''

  for index, value in df.iterrows():
    # When Building is full but address is na:
    if pd.notna(value['Building']) & pd.isna(value['Address']):

      df.loc[index, 'Full_Address'] = value['Building'] + ', ' + value['City'] + ', ' + value['State']
    # When Address is full but building is na:
    elif pd.notna(value['Address']) & pd.isna(value['Building']):
      df.loc[index, 'Full_Address'] = value['Address']+ ', ' + value['City'] + ', ' + value['State']

    else:
      df.loc[index, 'Full_Address'] = ''
      #Remove . at beginning
  df['Full_Address'] = df['Full_Address'].str.strip().str.replace(r'\(.*?\)', '', regex=True)

  return df
city_fulladr = full_address(city_post)
city_fulladr
city_fulladr.to_csv(f"files/{city}/{city}.csv", index=False)

In [ ]:
city_fulladr

,Page Number,Text,Company_Type,City,County,Company_Name,Address,Building,Full_Address
0,1,"KANSAS CITY , 248,381 Attorney",None,Kansas City,Missouri,"KANSAS CITY ,","248,381 Attorney",NaN,"248,381 Attorney, Kansas City, Missouri"
1,1,"JNO. A, EATON, Attorney, 800 U. 5.",Attorney,Kansas City,Missouri,"JNO. A, EATON, Attorney,",800 U. 5.,NaN,"800 U. 5., Kansas City, Missouri"
2,1,Mex. Trust building. Special attention,Attorney,Kansas City,Missouri,Mex. Trust building. Special attention,NaN,Trust building,"Trust building, Kansas City, Missouri"
3,1,given to Real Estate Ww,Attorney,Kansas City,Missouri,given to Real Estate Ww,NaN,NaN,
4,1,Real Estate,Real Estate,Kansas City,Missouri,Real Estate,NaN,NaN,
...,...,...,...,...,...,...,...,...,...
848,7,"L. R, Wright, 206 R. A. Long building.",Real Estate,Kansas City,Missouri,"L. R, Wright, 206 R. A. Long building.",NaN,206 R. A. Long building,"206 R. A. Long building, Kansas City, Missouri"
849,7,"M. B Wright, 919 Baltimore ave.",Real Estate,Kansas City,Missouri,"M. B Wright,",919 Baltimore ave.,NaN,"919 Baltimore ave., Kansas City, Missouri"
850,7,"Jamea R. Young, 335 Ridge pide.",Real Estate,Kansas City,Missouri,"Jamea R. Young,",335 Ridge pide.,NaN,"335 Ridge pide., Kansas City, Missouri"
851,7,"Zavala Land Inv. Co., 602 Grand Ave.",Real Estate,Kansas City,Missouri,"Zavala Land Inv. Co.,",602 Grand Ave.,NaN,"602 Grand Ave., Kansas City, Missouri"
